### Purpose - Take Weights and Chars Reports from Factset and convert into a Seismic Friendly flat file format.

In [17]:
# Import dependencies
import numpy as np
import pandas as pd
import xlsxwriter

### Step 1:  Wrangle and transform `Sector` data from FactSet

In [38]:
# Import sector weights tab and create df
df = pd.read_excel('input_template.xlsx', sheet_name = 'SECTOR WEIGHTS', skiprows=7, header= None, names=['Sector', 'SimPortWeight', 'IndexWeight'])

# Drop blank rows
df = df.dropna(axis=0)

# Delete Total Row
df = df[df['Sector'] != 'Total']

# Divide Weights by 100
df[['SimPortWeight', 'IndexWeight']] = df[['SimPortWeight', 'IndexWeight']].div(100)

# Import same file to wo skipping rows to get some add'l necessary meta_data
df_metadata = pd.read_excel('input_template.xlsx', sheet_name = 'SECTOR WEIGHTS')

# Get report date
rpt_date = df_metadata.iloc[4,0]

# Get report currency
rpt_curr = df_metadata.iloc[0,0]

# Get strategy code (allow for suffix such as EVE if exists)
code_raw = df_metadata.columns[0].split()[0]
if len(code_raw.split('_')) == 3:
    names = code_raw.split('_')[0:-1]
    rpt_code = names[0] + "_" + names[1]
else:
    rpt_code = code_raw.split('_')[0]

# Insert add'l cols needed for Seismic
df.insert(0, 'AsOfDate', rpt_date)
df.insert(1, 'StrategyCode', rpt_code)

# Preview df and metadata
print('Date: ', rpt_date, " ", "Date Type: ", type(rpt_date), ' Strategy Code: ', rpt_code, \
    ' Currency: ', rpt_curr)
df.head(13)

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

Date:  3/31/2021   Date Type:  <class 'str'>  Strategy Code:  MSMUA  Currency:  USD


,AsOfDate,StrategyCode,Sector,SimPortWeight,IndexWeight
0,3/31/2021,MSMUA,Communication Services,0.036840,0.092436
1,3/31/2021,MSMUA,Consumer Discretionary,0.095878,0.066099
2,3/31/2021,MSMUA,Consumer Staples,0.156847,0.110359
3,3/31/2021,MSMUA,Energy,0.000000,0.002510
4,3/31/2021,MSMUA,Financials,0.098769,0.089545
5,3/31/2021,MSMUA,Health Care,0.147171,0.188164
6,3/31/2021,MSMUA,Industrials,0.073050,0.077659
7,3/31/2021,MSMUA,Information Technology,0.292843,0.235251
8,3/31/2021,MSMUA,Materials,0.025790,0.028023
9,3/31/2021,MSMUA,Real Estate,0.047792,0.034429


### Step 2:  Wrangle and transform `Market Cap` data from FactSet

In [43]:
# Import mcap weights tab and create df
mcap = pd.read_excel('input_template.xlsx', sheet_name = 'MARKET CAP WEIGHTS', header = None, skiprows = 7, names= ('Label', 'SimWeight', 'IndexWeight'))

# Insert add'l cols needed for Seismic
mcap.insert(0, 'AsOfDate', rpt_date)
mcap.insert(1, 'StrategyCode', rpt_code)
mcap.insert(2, 'Currency', rpt_curr)

# Drop blank rows
mcap = mcap.dropna(axis=0)

# Delete Total Row
mcap = mcap[mcap['Label'] != 'Total']

# Insert a 'Range' col that maps ints for each bucket
#Length of mcap = 6, np.arrange gives an array starting at 0 and up to but not including length value.
range  = np.arange(len(mcap))
range += 1
mcap.insert(3, 'Range', range)

# Divide weights by 100
mcap[['SimWeight', 'IndexWeight']] = mcap[['SimWeight', 'IndexWeight']].div(100)
mcap

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

,AsOfDate,StrategyCode,Currency,Range,Label,SimWeight,IndexWeight
0,3/31/2021,MSMUA,USD,1,Over 100B,0.387585,0.370894
1,3/31/2021,MSMUA,USD,2,25B - 100B,0.387983,0.454373
2,3/31/2021,MSMUA,USD,3,15B - 25B,0.163761,0.104664
3,3/31/2021,MSMUA,USD,4,2B - 15B,0.060672,0.070069


In [44]:
# Import library and create currency symbols to be used in mkt cap output
import unicodedata

EUR_Code = '{}'.format(unicodedata.lookup("EURO SIGN"))
GBP_Code = '{}'.format(unicodedata.lookup('POUND SIGN'))
NZD_Code = 'NZD'
DKK_Code = 'DKK'
USD_Code = '$'
AUD_Code = 'A$'

In [45]:
# Create and run function to identify appropriate curr symbol for output

def currency_symbol(currency):
    if currency == 'USD':
        return USD_Code
    elif currency == 'GBP':
        return GBP_Code
    elif currency == 'DKK':
        return DKK_Code
    elif currency == 'NZD':
        return NZD_Code
    elif currency == 'AUD':
        return AUD_Code
    elif currency == 'EUR':
        return EUR_Code    

currency_sign = currency_symbol(rpt_curr)
currency_sign

'$'

In [46]:
# Input currency sign into mkt cap bucket categories

Mega_Cap = '> {0}100B'.format(currency_sign)
Big_Large_Cap = '{0}25B - {0}100B'.format(currency_sign)
Large_Cap = '{0}15B - {0}25B'.format(currency_sign)
Mid_Cap = '{0}2B - {0}15B'.format(currency_sign)
Small_Cap = '< {0}2B'.format(currency_sign)

In [47]:
# Replace each original mkt cap bucket to equivalent name w/ the curr sign

mcap['Label'] = mcap['Label'].replace('Over 100B', Mega_Cap)
mcap['Label'] = mcap['Label'].replace('25B - 100B', Big_Large_Cap)
mcap['Label'] = mcap['Label'].replace('15B - 25B', Large_Cap)
mcap['Label'] = mcap['Label'].replace('2B - 15B', Mid_Cap)
mcap['Label'] = mcap['Label'].replace('Under 2B', Small_Cap)

In [48]:
# Preview final mkt cap df
mcap

,AsOfDate,StrategyCode,Currency,Range,Label,SimWeight,IndexWeight
0,3/31/2021,MSMUA,USD,1,> $100B,0.387585,0.370894
1,3/31/2021,MSMUA,USD,2,$25B - $100B,0.387983,0.454373
2,3/31/2021,MSMUA,USD,3,$15B - $25B,0.163761,0.104664
3,3/31/2021,MSMUA,USD,4,$2B - $15B,0.060672,0.070069


### Step 3:  Wrangle and transform `Characteristics` data from FactSet 

In [49]:
# Import chars tab and create df
chars = pd.read_excel('input_template.xlsx', sheet_name = 'CHARACTERISTICS', skiprows= 8, skipfooter =2, header = None, names=['Char', 'PortValue', 'IndValue'])

# Insert add'l columns for Seismic
chars.insert(0, 'AsOfDate', rpt_date)
chars.insert(1, 'StrategyCode', rpt_code)
chars.insert(2, 'Currency', rpt_curr)

# Preview
chars.head()

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

,AsOfDate,StrategyCode,Currency,Char,PortValue,IndValue
0,3/31/2021,MSMUA,USD,# of Securities,79.000000,185.000000
1,3/31/2021,MSMUA,USD,P/E (1Yr Trailing),30.790772,31.490815
2,3/31/2021,MSMUA,USD,Dividend Yield,1.868637,2.032096
3,3/31/2021,MSMUA,USD,EPS Growth (5Yr Trailing),12.171331,11.923764
4,3/31/2021,MSMUA,USD,P/B,6.591903,6.474775


In [50]:
# Rename chars derived from FDS to Seismic desired names

chars['Char'] = chars['Char'].replace('# of Securities', 'Number of Securities')
chars['Char'] = chars['Char'].replace('P/E (1Yr Trailing)', 'Price/Earnings Ratio (LTM)')
chars['Char'] = chars['Char'].replace('Dividend Yield', 'Dividend Yield (Current)')
chars['Char'] = chars['Char'].replace('EPS Growth (5Yr Trailing)', 'EPS Growth (5 Yr. Historical)')
chars['Char'] = chars['Char'].replace('P/B', 'Price/Book Ratio')
chars['Char'] = chars['Char'].replace('Market Cap - Weighted Median', 'Weighted Median Market Cap')
chars['Char'] = chars['Char'].replace('Market Cap - Weighted Average', 'Weighted Average Market Cap')
chars

,AsOfDate,StrategyCode,Currency,Char,PortValue,IndValue
0,3/31/2021,MSMUA,USD,Number of Securities,79.000000,185.000000
1,3/31/2021,MSMUA,USD,Price/Earnings Ratio (LTM),30.790772,31.490815
2,3/31/2021,MSMUA,USD,Dividend Yield (Current),1.868637,2.032096
3,3/31/2021,MSMUA,USD,EPS Growth (5 Yr. Historical),12.171331,11.923764
4,3/31/2021,MSMUA,USD,Price/Book Ratio,6.591903,6.474775
5,3/31/2021,MSMUA,USD,Weighted Average Market Cap,101484.302640,146121.518055
6,3/31/2021,MSMUA,USD,Weighted Median Market Cap,62264.554688,61968.160156


In [51]:
# Transform numerical output to appropriate type

item_1 = chars.loc[5,'PortValue']
item_2 = chars.loc[6,'PortValue']
item_3 = chars.loc[5, 'IndValue']
item_4 = chars.loc[6, 'IndValue']
item_5 = chars.loc[2,'PortValue']
item_6 = chars.loc[2, 'IndValue']
item_7 = chars.loc[3, 'PortValue']
item_8 = chars.loc[3, 'IndValue']
item_9 = chars.loc[1, 'PortValue']
item_10 = chars.loc[1, 'IndValue']
item_11 = chars.loc[4, 'PortValue']
item_12 = chars.loc[4, 'IndValue']

def MCap(item):
    item /= 1000
    item = item.round(1)
    item = str(item)
    item = currency_sign + item + ' B'
    return item

def PercString(item):
    item = "{:0.2f}%".format(item)
    return item

def RoundString(item):
    item = "{:0.2f}".format(item)
    return item

item_1 = MCap(item_1)
item_2 = MCap(item_2)
item_3 = MCap(item_3)
item_4 = MCap(item_4)
item_5 = PercString(item_5)
item_6 = PercString(item_6)
item_7 = PercString(item_7)
item_8 = PercString(item_8)
item_9 = RoundString(item_9)
item_10 = RoundString(item_10)
item_11 = RoundString(item_11)
item_12 = RoundString(item_12)


chars.loc[5,'PortValue'] = item_1
chars.loc[6,'PortValue'] = item_2 
chars.loc[5,'IndValue'] = item_3 
chars.loc[6, 'IndValue'] = item_4
chars.loc[2,'PortValue'] = item_5
chars.loc[2, 'IndValue'] = item_6
chars.loc[3, 'PortValue'] = item_7
chars.loc[3, 'IndValue'] = item_8
chars.loc[1, 'PortValue'] = item_9
chars.loc[1, 'IndValue'] = item_10
chars.loc[4, 'PortValue'] = item_11
chars.loc[4, 'IndValue'] = item_12

chars

,AsOfDate,StrategyCode,Currency,Char,PortValue,IndValue
0,3/31/2021,MSMUA,USD,Number of Securities,79.0,185.0
1,3/31/2021,MSMUA,USD,Price/Earnings Ratio (LTM),30.79,31.49
2,3/31/2021,MSMUA,USD,Dividend Yield (Current),1.87%,2.03%
3,3/31/2021,MSMUA,USD,EPS Growth (5 Yr. Historical),12.17%,11.92%
4,3/31/2021,MSMUA,USD,Price/Book Ratio,6.59,6.47
5,3/31/2021,MSMUA,USD,Weighted Average Market Cap,$101.5 B,$146.1 B
6,3/31/2021,MSMUA,USD,Weighted Median Market Cap,$62.3 B,$62.0 B


### Step 4:  Wrangle and transform `Top Bottom 10` data from FactSet

In [55]:
# Import Top 10 Portfolio Holdings by Weight tab and create df

tb = pd.read_excel('input_template.xlsx', sheet_name = 'TOP 10 WEIGHTS', header = None, skiprows = 8, skipfooter = 1, names=['Holding', 'Weight'])

# Drop blank rows
tb = tb.dropna(axis=0)

# Delete Total Row
tb = tb[tb['Holding'] != 'Total']

# Sum up total of the top 10 holdings weights
ten_wgt_sum = tb['Weight'].sum()

# Insert add'l columns need for Seismic
tb.insert(0, 'AsOfDate', rpt_date)
tb.insert(1, 'StrategyCode', rpt_code)
tb.insert(4, 'WeightSum', ten_wgt_sum)

# Divide Weights by 100
tb[['Weight','WeightSum']] = tb[['Weight', 'WeightSum']].div(100)

tb

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:/Characteristics/Flat File/Template/Flat_File_Template.xlsx'

,AsOfDate,StrategyCode,Holding,Weight,WeightSum
0,3/31/2021,MSMUA,Accenture Plc Class A,0.058107,0.343337
1,3/31/2021,MSMUA,Progressive Corporation,0.046811,0.343337
2,3/31/2021,MSMUA,Bristol-Myers Squibb Company,0.040567,0.343337
3,3/31/2021,MSMUA,Crown Castle International Corp,0.037480,0.343337
4,3/31/2021,MSMUA,International Business Machines Corporation,0.031456,0.343337
5,3/31/2021,MSMUA,Johnson & Johnson,0.026894,0.343337
6,3/31/2021,MSMUA,"McCormick & Company, Incorporated",0.026474,0.343337
7,3/31/2021,MSMUA,Target Corporation,0.026103,0.343337
8,3/31/2021,MSMUA,"Citrix Systems, Inc.",0.024796,0.343337
9,3/31/2021,MSMUA,Campbell Soup Company,0.024648,0.343337


### Step 5:  Output to on single excel file

In [56]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('output_file.xlsx', engine = 'xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sector', index=False)
mcap.to_excel(writer, sheet_name='MCap', index=False)
chars.to_excel(writer, sheet_name='Chars', index = False)
tb.to_excel(writer, sheet_name='TB', index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook = writer.book
worksheet_1 = writer.sheets['Sector']
worksheet_2 = writer.sheets['MCap']
worksheet_3 = writer.sheets['Chars']
worksheet_4 = writer.sheets['TB']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '0.00%'})
format2 = workbook.add_format()

worksheet_1.set_column('D:E', 18 , format1)
worksheet_1.set_column('A:C', 24, format2)
worksheet_2.set_column('F:G', 18 , format1)
worksheet_2.set_column('A:E', 18, format2)
worksheet_3.set_column('D:D', 30, format2)
worksheet_3.set_column('A:C',18, format2)
worksheet_3.set_column('E:F',18, format2)
worksheet_4.set_column('A:C', 18, format2)
worksheet_4.set_column('D:E', 18, format1)

writer.save()

# Use 'filepath' below if want to make sharable with IB and MM, can use PA directory for input and output.
#filepath 'M:\Characteristics\Flat File\Output\Flat_File_Chars_Weights.xlsx'